In [1]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='dataset/Cora', name='Cora')
data = dataset[0]

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing...
Done!


In [6]:
import louvain
import igraph as ig
import numpy as np
import pandas as pd
from natsort import natsorted

g = ig.Graph(directed=False)
g.add_vertices(data.edge_index.shape[1])  # this adds adjacency.shape[0] vertices
g.add_edges(list(zip(data.edge_index[0], data.edge_index[1])))

part = louvain.find_partition(
    g,
    louvain.RBConfigurationVertexPartition,
    resolution_parameter=1.0,
    seed=2022,
)
groups = np.array(part.membership)
result = pd.Categorical(
    values=groups.astype('U'),
    categories=natsorted(map(str, np.unique(groups))),
)